# First steps

This is an example of what you can already with GMT/Python.

Modules are available as functions in the `gmt` package. See the [API Reference](api.html) for a list of all available functions.

Start by importing the package:

In [ ]:
import gmt

We'll reproduce an example from the [GMT Tutorial](http://gmt.soest.hawaii.edu/doc/latest/GMT_Tutorial.html#session-one).

The Python functions accept the same command-line options as the GMT modules.

In [ ]:
# Start a new figure. 
gmt.figure()
# Creat a global basemap with the Mercator projection
gmt.psbasemap(R='130/150/35/50', J='M6i', B=True, P=True)
# Plot the quake data from the GMT tutorial
gmt.psxy('@tut_quakes.ngdc', S='c0.4c', G='red', W='faint', 
         i='4,3')
# Unlike the GMT command-line interface, no figure 
# file is generated unless savefig or psconvert are 
# called.
gmt.psconvert(F='myfigure', T='G', A=True)

Insert the figure into the Jupyter notebook.

In [ ]:
# Display the figure in the notebook
from IPython.display import Image, display
Image(filename='myfigure.png', embed=True, width=500)

You can also use longer aliases for the command-line options to make your code more Pythonic. The docstrings for each module list the possible aliases:

In [ ]:
help(gmt.psxy)

Lets make the same figure but using the available aliases.

In [ ]:
gmt.figure()
# Arguments like R can also receive input as lists.
# This makes it easier to generate such arguments 
# programatically
gmt.psbasemap(region=[130, 150, 35, 50], projection='M6i', 
              frame=True, portrait=True)
# You can mix using aliases and the command-line names
gmt.psxy('@tut_quakes.ngdc', style='c0.4c', G='red', 
         pen='faint', i='4,3')
gmt.psconvert(prefix='myotherfigure', fmt='G', crop=True)

Image(filename='myotherfigure.png', embed=True, width=500)

In [ ]:
# Clean up the generated files
import os
os.remove('myfigure.png')
os.remove('myotherfigure.png')